# Imports


In [2]:
import pandas as pd
import requests
import json
import html
from datetime import date
import csv

In [ ]:
filename ="companies.csv"
 
# opening the file using "with"
# statement
with open(filename, 'r') as data:
  companies={row[0]:row[1] for row in csv.reader(data)}


In [ ]:
tokens = set(companies.keys())

## Check for bad job tokens

In [ ]:
# No output == good
# Output == company api not found
'''
for company in tokens:
    if not requests.get(f'https://boards-api.greenhouse.io/v1/boards/{company}/jobs'):
      print(company)
      
      '''

"\nfor company in tokens:\n    if not requests.get(f'https://boards-api.greenhouse.io/v1/boards/{company}/jobs'):\n      print(company)\n      \n      "

## Job criteria
Finds job titles that match criteria.    
Roles is for job role (engineer, developer, etc), level is for expereince level keywords ('junior', 'associate',etc), exclude is for tersm you don't want ('senior', etc.). Should be lower case.

In [ ]:
roles = {'developer','engineer', 'frontend', 'software', 'apprentice', 'apprenticeship', 'front-end', 'backend', 'back-end', 'jr.', 'jr' }
# Includes 'software' for titles that are just 'software enginer', etc.
levels = {'junior', 'entry-level', 'grad', 'graduate', 'apprentice', 'apprenticeship', 'software', 'entry', 'intern', 'i', '1', 'associate'}
# Optional, but helps exclude higher level postiions
exclude = {'senior', 'principal' , 'sr.', 'ii', 'iii' }

# Main Code
Finds jobs and outputs results as csv or excel file

In [ ]:
# Return a list of elligble jobs from a set of companies
def find_jobs(companies : dict, roles : set , levels : set , exclude : set = {}) -> list:
  
  tokens = set(companies.keys())
  results = []

  for company in tokens:
    res=requests.get(f'https://boards-api.greenhouse.io/v1/boards/{company}/jobs')
    if res:
      jobs=json.loads(res.text).get('jobs')

      if jobs:
        for job in jobs:
            title=set(job.get('title').lower().split())
            if title.intersection(roles) and title.intersection(levels) and not title.intersection(exclude) :
              job['token'] = company
              job['company'] = companies.get(company)
              results.append(job)

  return results

  

In [ ]:
# Get job details for elligble jobs and return a dataframe with the job data
def get_details(results: list ) -> list: 
  data = []
  for job in results:
    url = f'https://boards-api.greenhouse.io/v1/boards/{job.get("token")}/jobs/{job.get("id")}'
    res = requests.get(url)

    if res:
      job_detail = json.loads(res.text)
      job_info = {'title': job_detail.get('title'), 'company' : job.get('company'), 'link': job_detail.get('absolute_url'), 
                  'description' : html.unescape(job_detail.get('content')), 'date' : job_detail.get('updated_at'), 
                  'remote' : None, 'greenhouse_id' : job.get('id'), 'greenhouse_api_url' : url }
      
      if job_detail.get('offices'):
        job_info['location'] = str(job_detail.get('offices'))
        offices = {office.get('name').lower() for office in job_detail.get('offices')}
        if 'remote' in offices:
          job_info['remote'] = True
        else:
          job_info['remote'] = False
          
      data.append(job_info)

  return data

In [ ]:
# Driver code

results = find_jobs(companies, roles, levels, exclude)

data = get_details(results)

df = pd.DataFrame()
df = df.from_records(data)


## Job links

In [ ]:
f'Found {len(df.link)} jobs!'

'Found 226 jobs!'

In [ ]:
# Job links
df.link

0      https://www.tempus.com/careers/job/?gh_jid=656...
1      https://www.tempus.com/careers/job/?gh_jid=436...
2      https://www.tempus.com/careers/job/?gh_jid=656...
3      https://www.tempus.com/careers/job/?gh_jid=666...
4      https://www.tempus.com/careers/job/?gh_jid=666...
                             ...                        
221    https://boards.greenhouse.io/liveperson/jobs/4...
222    https://boards.greenhouse.io/imprint/jobs/4722...
223             https://ample.com/jobs?gh_jid=4000768005
224             https://ample.com/jobs?gh_jid=4224193005
225             https://ample.com/jobs?gh_jid=4169834005
Name: link, Length: 226, dtype: object